# UnderstandMD: Translating Clinical Language into Plain English with Generative AI

This project explores the use of a generative AI model to convert clinical medical descriptions into plain-language explanations understandable by a 12-year-old. The goal is to support health literacy, reduce confusion, and make health information more accessible to the general public.

In [3]:
# === Core Libraries ===
import os
import time
import tempfile
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
model_path = os.getenv("LLAMA_MODEL_PATH")
print("Loaded model path:", model_path)

# === Data & Utility Libraries ===
import pandas as pd
from IPython.display import display
from tqdm import tqdm

# === Audio Processing ===
from pydub import AudioSegment

# === Text-to-Speech (Offline) ===
import pyttsx3
from scipy.io.wavfile import read

# === Gradio Interface ===
import gradio as gr

# === Fuzzy Matching ===
from rapidfuzz import process

# === Local LLM: Mistral 7B with llama-cpp ===
from llama_cpp import Llama

# Validate model path
if not model_path or not os.path.isfile(model_path):
    raise FileNotFoundError(
        f"Model file not found.\n\n"
        f"Expected at: {model_path}\n"
        f"Tip: Check your `.env` file in `understandmd/.env` and verify that LLAMA_MODEL_PATH is set correctly."
    )

# Load the Llama model
llm = Llama(
    model_path=model_path,
    n_ctx=2048,
    n_threads=8,
    n_batch=512,
    verbose=False
)

Loaded model path: /Users/victoriablante/1-MSDS/summer25/understandmd/models/mistral-7b-instruct-v0.1.Q3_K_M.gguf


llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h96 

## Project Introduction

Medical information is often written in complex, technical language that can be difficult for patients to understand. This communication gap can lead to confusion, reduced health literacy, and poorer health outcomes, especially for those without medical backgrounds.

This project aims to bridge that gap by using a locally hosted generative AI model to translate clinical descriptions into simple, easy-to-understand explanations suitable for a 12-year-old. It also highlights common symptoms and converts the explanation into spoken audio using text-to-speech, making the content more accessible to users with reading difficulties, visual impairments, or different learning preferences.

By leveraging the Mistral-7B-Instruct model, we ensure that all processing happens privately and cost-free without relying on cloud-based APIs. The result is a reproducible, user-friendly system that supports health literacy and empowers patients to better understand their own care.

## Data Description

This project uses two complementary datasets to support the generation and evaluation of plain-language medical explanations:

### 1. **Manually Curated Dataset (50 entries)**  
This dataset was created using public clinical descriptions from the **Mayo Clinic** website. Each row includes:
- `condition`: The name of the medical condition  
- `clinical_description`: A technical or clinical explanation of the condition  
- `simple_explanation`: A human-written, gold-standard explanation in plain language (optional)

This dataset serves as a benchmark for prompt testing and model evaluation.  
*Limitations:* Only a subset of conditions includes gold-standard outputs, and the complexity of descriptions varies.

---

### 2. **Scraped Dataset (MedlinePlus)**  
To expand coverage, medical condition pages were scraped from [MedlinePlus](https://medlineplus.gov/), a trusted health resource maintained by the U.S. National Library of Medicine. This dataset includes:
- `term`: The condition name  
- `description`: A brief clinical summary extracted from each page

This larger dataset supports real-time term matching, including for slightly misspelled inputs.  
*Limitations:* Descriptions may be brief and vary in specificity or clarity.

---

Together, these datasets enable both robust prompt engineering and wide user-facing coverage, helping ensure that even unfamiliar or imperfect terms can return accurate, readable explanations.

In [ ]:
clinical_df = pd.read_csv('clinical_conditions.csv')
clinical_df.head(2)

In [3]:
medical_df = pd.read_csv("medline_conditions.csv")
known_terms = medical_df['term'].dropna().str.lower().tolist()

## Data Preprocessing

Before generating explanations, we performed light preprocessing to clean and standardize the dataset:

- Removed newline characters from clinical descriptions
- Collapsed extra spaces into single spaces
- Stripped leading and trailing whitespace from all text fields

This helped ensure that the input text fed into the language model was consistent and clean, minimizing formatting artifacts that could affect generation quality.


In [ ]:
def clean_text(text):
    # Remove newlines, strip leading/trailing spaces, and collapse double spaces
    return ' '.join(text.replace('\n', ' ').split())

clinical_df['clinical_description'] = clinical_df['clinical_description'].apply(clean_text)

clinical_df['condition'] = clinical_df['condition'].str.strip()
clinical_df['clinical_description'] = clinical_df['clinical_description'].str.strip()

print(f"Dataset ready. Total rows: {len(clinical_df)}")

## Model Implementation

We used the Mistral-7B-Instruct model in GGUF format, hosted locally with the `llama-cpp-python` library. This instruction-tuned model is well-suited for tasks like "Explain in simple terms," making it ideal for translating clinical language into plain English.

The model was run fully offline on an Apple M3 Pro chip using 8 threads and a quantized 4-bit version to optimize for CPU performance. This setup allows us to generate responses efficiently without relying on API keys, cloud services, or paid infrastructure.

Running the model locally also ensures user privacy and makes the system fully reproducible for anyone with the required hardware.

### Test Prompt

In [ ]:
test_prompt = """You are a health explainer. Translate the following medical term into plain English so a 12-year-old can understand it.

Term: Atrial fibrillation
Description: Atrial fibrillation is an irregular and often very rapid heart rhythm. It can lead to blood clots, stroke, heart failure, and other complications.

Plain Explanation:"""

output = llm(test_prompt, max_tokens=150, stop=["\n\n"])
print(output["choices"][0]["text"].strip())

## Prompt Engineering and Methodology

I designed structured prompts that ask the model to output both a plain-language explanation and a list of symptoms. The system handles cases where a description is provided, as well as when only a term is available.

The format uses role-based chat messages to guide the model and ensures consistent output formatting for downstream processing.

### Prompt

In [6]:
# === Fuzzy match input ===
def match_term(user_input, threshold=90):
    result = process.extractOne(user_input, known_terms)
    if result:
        match, score, _ = result
        if score >= threshold:
            return match
    return None

In [7]:
def run_llm_with_mistral(prompt, max_tokens=280):
    messages = [
        {"role": "system", "content": "You are a health explainer who explains medical conditions to 12-year-olds."},
        {"role": "user", "content": prompt}
    ]
    try:
        result = llm.create_chat_completion(messages=messages, max_tokens=max_tokens)
        response_text = result["choices"][0]["message"]["content"]
        return {"choices": [{"text": response_text}]}
    except Exception as e:
        print(f"[ERROR] LLM generation failed: {e}")
        return {"choices": [{"text": ""}]}

In [8]:
def generate_explanation_then_symptoms(term, desc, llm, max_tokens=280):
    if desc and desc.strip():  # Use the description if available
        prompt = f"""You are a health explainer. Given the medical term and its description, do two things:

1. Explain the condition so a 12-year-old can understand it.
2. Then list **up to 4 common symptoms** in simple language using bullet points.

Format your response like this:

Plain Explanation: ...
Symptoms:
- ...
- ...
- ...
- ...

Term: {term}
Description: {desc}

Your response:"""
    else:
        prompt = f"""You are a health explainer. Given a medical term, do two things:

1. Explain the condition so a 12-year-old can understand it.
2. Then list **up to 4 common symptoms** in simple language using bullet points.

Format your response like this:

Plain Explanation: ...
Symptoms:
- ...
- ...
- ...
- ...

Term: {term}

Your response:"""

    try:
        response = llm(prompt, max_tokens=max_tokens)
        return response["choices"][0]["text"].strip()
    except Exception as e:
        return f"Error: {e}"

In [2]:
def split_explanation_and_symptoms(text):
    if "Symptoms:" in text:
        explanation, symptoms = text.split("Symptoms:", 1)
        explanation = explanation.replace("Plain Explanation:", "").strip()
        symptoms = symptoms.strip()
    else:
        explanation = text.strip()
        symptoms = ""
    return explanation, symptoms

explanations = []
symptom_lists = []

for i, row in tqdm(clinical_df.iterrows(), total=len(clinical_df)):
    term = row["condition"]  # or "term"
    desc = row["clinical_description"]
    full_text = generate_explanation_then_symptoms(term, desc, llm, max_tokens=280)
    explanation, symptoms = split_explanation_and_symptoms(full_text)
    explanations.append(explanation)
    symptom_lists.append(symptoms)

clinical_df["plain_explanation"] = explanations
clinical_df["symptoms"] = symptom_lists
clinical_df.to_csv("understandmd_generated.csv", index=False)


## Experiments and Results

I tested the model on a set of 50 medical conditions. For each one, the system generated a plain-language explanation and a list of symptoms. The responses were consistently understandable and aligned well with a 12-year-old reading level.

Below are selected examples that illustrate how the model translates clinical language into accessible explanations.

In [ ]:
new_df = pd.read_csv('understandmd_generated.csv')
# Ensure full text is shown in notebook output
pd.set_option('display.max_colwidth', None)

In [ ]:
# Display a few example rows
example_rows = new_df[['condition', 'clinical_description', 'plain_explanation', 'symptoms']].iloc[[0, 5, 22, 37]]
display(example_rows)

## Text-to-Speech Generation

Medical explanations are not always accessible to individuals with visual impairments, low literacy, or different learning preferences. To make the output more inclusive, we convert plain-language explanations into natural-sounding audio using the `pyttsx3` text-to-speech library, which works fully offline.

We format the model’s output into a complete sentence that combines the explanation and a list of symptoms in a natural way. The result is saved as a `.wav` file that can be played directly, making the information easier to access for a wide range of users.


In [14]:
def prepare_speech_text(explanation, symptoms):
    # Clean up symptom bullets and join them naturally
    if symptoms:
        symptoms_list = [line.strip("- ").strip() for line in symptoms.split("\n") if line.strip()]
        if len(symptoms_list) == 1:
            symptom_str = symptoms_list[0]
        else:
            symptom_str = ", ".join(symptoms_list[:-1]) + ", and " + symptoms_list[-1]
        speech = f"{explanation} Symptoms include: {symptom_str}."
    else:
        speech = explanation
    return speech


def speak_with_pyttsx3(text, rate=150):
    engine = pyttsx3.init()
    engine.setProperty("rate", rate)

    # Save as AIFF
    with tempfile.NamedTemporaryFile(delete=False, suffix=".aiff") as aiff_file:
        aiff_path = aiff_file.name

    engine.save_to_file(text, aiff_path)
    engine.runAndWait()

    # Wait a moment to ensure file is written (esp. on macOS)
    time.sleep(0.5)

    if not os.path.exists(aiff_path) or os.path.getsize(aiff_path) < 1000:
        raise RuntimeError("AIFF file not written or empty")

    # Convert to proper WAV format
    sound = AudioSegment.from_file(aiff_path, format="aiff")
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as wav_file:
        wav_path = wav_file.name
    sound.export(wav_path, format="wav")

    # Load WAV
    sr, audio_np = read(wav_path)
    if audio_np.size == 0:
        raise RuntimeError("WAV file exists but is silent")
    return (sr, audio_np)

## Full Inference Pipeline

The pipeline performs three key tasks:

1. **Fuzzy match** the user input to known medical terms.
2. **Generate** a plain-language explanation and list of symptoms using the LLM.
3. **Convert** the explanation to audio using text-to-speech.

This pipeline ensures users receive accessible, understandable, and spoken explanations of medical terms with no need for internet access or external APIs.

In [16]:
def full_pipeline(user_input, desc):
    term = match_term(user_input) or user_input
    raw_response = generate_explanation_then_symptoms(term, desc, run_llm_with_mistral)
    explanation, symptoms = split_explanation_and_symptoms(raw_response)
    # NEW: Combine explanation + symptoms into one spoken paragraph
    speech_text = prepare_speech_text(explanation, symptoms)
    try:
        audio = speak_with_pyttsx3(speech_text, rate=150)
    except Exception as e:
        audio = None

    return explanation, symptoms, audio

## Interactive Interface

To make the system user-friendly and accessible to a non-technical audience, we built an interactive interface using Gradio.

The interface includes:

- A **text input** for the user to enter a medical term.
- An optional field to provide additional **clinical context**.
- A **"Generate Explanation"** button styled in soft blue for clarity and visibility.
- Output sections that display:
  - A plain-language **explanation**,
  - A list of **common symptoms**,
  - And a **playable audio file** of the explanation.

The interface uses custom CSS for a clean, branded look and responds to both button clicks and Enter key submissions.

This no-code UI allows anyone to interact with the generative model without needing to install libraries, write code, or understand AI—making it an ideal tool for improving health communication at scale.

In [ ]:
# blue colors
soft_blue = "#6366f1"
light_blue_bg = "rgba(59, 130, 246, 0.1)"

# Custom CSS to override the Soft theme's blue label pills
custom_css = f"""
/* Title color */
h1 {{
    color: {soft_blue} !important;
    font-family: 'Segoe UI', sans-serif;
    text-align: center;
    margin-bottom: 24px;
}}

/* Button styling */
#generate-button {{
    background-color: {light_blue_bg} !important;
    color: {soft_blue} !important;
    font-weight: bold !important;
    font-size: 16px !important;
    border: 2px solid {soft_blue} !important;
    border-radius: 14px !important;
    padding: 14px 24px !important;
    width: 100%;
    text-align: center;
    box-shadow: 0px 2px 6px rgba(0,0,0,0.1);
    transition: background-color 0.3s ease;
}}

#generate-button:hover {{
    background-color: rgba(79, 70, 229, 0.15) !important;
}}
"""
# Gradio app
with gr.Blocks(theme=gr.themes.Soft(), css=custom_css) as demo:
    gr.Markdown("# UnderstandMD: Medical Terms Made Simple")

    with gr.Row():
        input_box = gr.Textbox(label="Medical Term", placeholder="e.g., asthma", scale=2)
        desc_box = gr.Textbox(label="(Optional) Add a Description", placeholder="Add any specific context")

    submit_btn = gr.Button("Generate Explanation", elem_id="generate-button")

    with gr.Column():
        output_text = gr.Textbox(label="Plain Explanation")
        output_symptoms = gr.Textbox(label="Symptoms")
        output_audio = gr.Audio(label="Listen", type="filepath")

    submit_btn.click(fn=full_pipeline, inputs=[input_box, desc_box], outputs=[output_text, output_symptoms, output_audio])
    input_box.submit(fn=full_pipeline, inputs=[input_box, desc_box], outputs=[output_text, output_symptoms, output_audio])

demo.launch()

## Conclusion

This project shows that a locally hosted generative AI model can successfully simplify complex medical language. Using Mistral-7B-Instruct and carefully designed prompts, we translated clinical descriptions into plain-language explanations with minimal errors.

To further enhance accessibility, I added a text-to-speech component using `pyttsx3`, allowing users to listen to the explanations. This supports individuals who prefer auditory information or may struggle with reading medical content.

The resulting system is cost-free, privacy-preserving, and easily extendable. Future improvements could include multilingual support, full offline functionality, integration into patient-facing tools, or evaluation with real user feedback.

Overall, this work highlights the potential of generative AI to make healthcare communication more understandable and inclusive.